<a href="https://colab.research.google.com/github/SMN-SENAMHI/S2S/blob/main/Humedad_relativa_2m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies required for this example.
# If you're not using pip to install depdencies (for example, you're using conda or uv),
# skip this cell and and install using the package manager of your choice.
# Restart the notebook after installing dependencies.
%pip install "xarray[complete]>=2025.1.2" "zarr>=3.0.4" requests aiohttp
!apt-get install -y libproj-dev proj-data proj-bin libgeos-dev
!pip install cython
!pip install cartopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.4/205.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.4/259.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Do

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import pandas as pd
import numpy as np
import tempfile
import requests
import os

# Descargar shapefiles
shapefiles = {
    "departamentos.shp": "https://raw.githubusercontent.com/SMN-SENAMHI/Materiales/main/shape/DEPARTAMENTOS.shp",
    "departamentos.dbf": "https://raw.githubusercontent.com/SMN-SENAMHI/Materiales/main/shape/DEPARTAMENTOS.dbf",
    "departamentos.shx": "https://raw.githubusercontent.com/SMN-SENAMHI/Materiales/main/shape/DEPARTAMENTOS.shx",
}
shapefiles_extra = {
    "SUDA_NUEVO_SIN_PERU_3.shp": "https://raw.githubusercontent.com/SMN-SENAMHI/Materiales/main/shape/SUDA_NUEVO_SIN_PERU_3.shp",
    "SUDA_NUEVO_SIN_PERU_3.dbf": "https://raw.githubusercontent.com/SMN-SENAMHI/Materiales/main/shape/SUDA_NUEVO_SIN_PERU_3.dbf",
    "SUDA_NUEVO_SIN_PERU_3.shx": "https://raw.githubusercontent.com/SMN-SENAMHI/Materiales/main/shape/SUDA_NUEVO_SIN_PERU_3.shx",
}

tempdir = tempfile.mkdtemp()
for files in [shapefiles, shapefiles_extra]:
    for filename, url in files.items():
        response = requests.get(url)
        with open(os.path.join(tempdir, filename), "wb") as f:
            f.write(response.content)

gdf_peru = gpd.read_file(os.path.join(tempdir, "departamentos.shp"))
gdf_suda = gpd.read_file(os.path.join(tempdir, "SUDA_NUEVO_SIN_PERU_3.shp"))

# Abrir dataset
ds = xr.open_zarr(
    "https://data.dynamical.org/noaa/gefs/forecast-35-day/latest.zarr?email=optional@email.com",
    decode_timedelta=True
)
latest_init = ds.init_time.max().values
print("Usando init_time:", pd.to_datetime(latest_init).strftime("%Y-%m-%d"))

ds_sub = ds.sel(init_time=latest_init).sel(latitude=slice(0.5, -19.5), longitude=slice(-82, -68))
ds_members = ds_sub.sel(lead_time=slice("0h", "27d"))

# HUMEDAD RELATIVA
rh_all = ds_members["relative_humidity_2m"]
forecast_time = ds_members.init_time + ds_members.lead_time
rh_all = rh_all.assign_coords(forecast_time=forecast_time)
rh_all = rh_all.swap_dims({"lead_time": "forecast_time"})
rh_daily_all = rh_all.groupby("forecast_time.date").mean(dim="forecast_time")

semanas = {
    "SEMANA 1": slice(0, 7),
    "SEMANA 2": slice(7, 14),
    "SEMANA 3": slice(14, 21),
    "SEMANA 4": slice(21, 28),
}

# Colores para humedad relativa
cmap_rh = plt.cm.YlGnBu
norm_rh = mcolors.Normalize(vmin=0, vmax=100)

extent = [-82, -68, -19.5, 0.5]
x_ticks = range(-82, -67, 3)
y_ticks = range(-20, 1, 5)

# Graficar por miembro
members = rh_all.ensemble_member.values
for member in members:
    fig, axes = plt.subplots(1, 4, figsize=(20, 6), subplot_kw={'projection': ccrs.PlateCarree()}, dpi=300)
    fig.subplots_adjust(wspace=0.01, hspace=0)

    for i, (ax, (semana, rango)) in enumerate(zip(axes, semanas.items())):
        rh_member = rh_daily_all.sel(ensemble_member=member)
        rh_avg = rh_member.isel(date=rango).mean(dim="date")
        week_dates = rh_member.date.values[rango]
        start_date = pd.to_datetime(str(week_dates[0])).strftime("%d-%b").upper()
        end_date = pd.to_datetime(str(week_dates[-1])).strftime("%d-%b").upper()

        ax.set_extent(extent, crs=ccrs.PlateCarree())
        ax.set_xticks(x_ticks, crs=ccrs.PlateCarree())
        ax.set_yticks(y_ticks, crs=ccrs.PlateCarree())
        ax.tick_params(labelbottom=True, labelleft=(i == 0))

        ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='black')
        ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
        ax.add_feature(cfeature.LAND, facecolor='lightgray')

        mesh = ax.pcolormesh(rh_avg.longitude, rh_avg.latitude, rh_avg,
                             cmap=cmap_rh, norm=norm_rh, shading="nearest")
        gdf_peru.boundary.plot(ax=ax, edgecolor='black', linewidth=0.5, zorder=5)
        gdf_suda.plot(ax=ax, color='white', linewidth=0, zorder=10)

        ax.set_title(f"**{semana}**\n{start_date} – {end_date}", fontsize=11)

    cbar = fig.colorbar(mesh, ax=axes.ravel().tolist(), orientation="vertical", shrink=0.7, pad=0.03)
    cbar.set_label("Humedad Relativa [%]")

    plt.suptitle(f"RH SEMANAL - MIEMBRO {member} [GEFS]", fontsize=16, y=1.07)
    output_file = f"rh_miembro_{member}_semanal_peru_{pd.to_datetime(latest_init).strftime('%Y%m%d')}.png"
    plt.savefig(output_file, bbox_inches='tight', pad_inches=0.05)
    plt.close()
    print(f"Imagen guardada como: {output_file}")

# Promedio de ensemble
fig, axes = plt.subplots(1, 4, figsize=(20, 6), subplot_kw={'projection': ccrs.PlateCarree()}, dpi=300)
fig.subplots_adjust(wspace=0.01, hspace=0)
rh_prom = rh_daily_all.mean(dim="ensemble_member")

for i, (ax, (semana, rango)) in enumerate(zip(axes, semanas.items())):
    rh_avg = rh_prom.isel(date=rango).mean(dim="date")
    week_dates = rh_prom.date.values[rango]
    start_date = pd.to_datetime(str(week_dates[0])).strftime("%d-%b").upper()
    end_date = pd.to_datetime(str(week_dates[-1])).strftime("%d-%b").upper()

    ax.set_extent(extent, crs=ccrs.PlateCarree())
    ax.set_xticks(x_ticks, crs=ccrs.PlateCarree())
    ax.set_yticks(y_ticks, crs=ccrs.PlateCarree())
    ax.tick_params(labelbottom=True, labelleft=(i == 0))

    ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='black')
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')

    mesh = ax.pcolormesh(rh_avg.longitude, rh_avg.latitude, rh_avg,
                         cmap=cmap_rh, norm=norm_rh, shading="nearest")
    gdf_peru.boundary.plot(ax=ax, edgecolor='black', linewidth=0.5, zorder=5)
    gdf_suda.plot(ax=ax, color='white', linewidth=0, zorder=10)

    ax.set_title(f"**{semana}**\n{start_date} – {end_date}", fontsize=11)

cbar = fig.colorbar(mesh, ax=axes.ravel().tolist(), orientation="vertical", shrink=0.7, pad=0.03)
cbar.set_label("Humedad Relativa [%]")

plt.suptitle("RH SEMANAL PROMEDIO DE ENSEMBLES PARA PERÚ [GEFS]", fontsize=16, y=1.07)
output_file = f"rh_ensmean_semanal_peru_{pd.to_datetime(latest_init).strftime('%Y%m%d')}.png"
plt.savefig(output_file, bbox_inches='tight', pad_inches=0.05)
plt.show()
print(f"Imagen guardada como: {output_file}")


Usando init_time: 2025-06-10


/usr/local/lib/python3.11/dist-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/10m_physical/ne_10m_land.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
/usr/local/lib/python3.11/dist-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/10m_cultural/ne_10m_admin_0_boundary_lines_land.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
/usr/local/lib/python3.11/dist-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/10m_physical/ne_10m_coastline.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)


Imagen guardada como: rh_miembro_0_semanal_peru_20250610.png
Imagen guardada como: rh_miembro_1_semanal_peru_20250610.png
Imagen guardada como: rh_miembro_2_semanal_peru_20250610.png
Imagen guardada como: rh_miembro_3_semanal_peru_20250610.png
Imagen guardada como: rh_miembro_4_semanal_peru_20250610.png
Imagen guardada como: rh_miembro_5_semanal_peru_20250610.png
